In [4]:
from ultralytics import YOLO
import cv2
import torch
import time
import numpy as np
from transformers import pipeline
from PIL import Image
import dist_measurement

yolov8_path = '../../models/object-detection/v1/yolov8m.pt'
wotr_path = '../../models/object-detection/v3/best_v3.pt'

# model should pay attention to these class indexes
wotr_class_pruned_indexes = [0, 1, 2, 3, 4, 12, 13, 14, 15, 16, 20, 21]

# for these indexes, we may not measure distance
wotr_class_important_indexes = [3, 4, 20, 21]

model_yolov8 = None
model_wotr = None
model_midas = None
model_captioning = None
transform_midas = None
device_midas = None

def initialize_models():
    global model_yolov8
    global model_wotr
    global model_midas
    global model_captioning
    global transform_midas
    global device_midas

    model_yolov8 = YOLO(yolov8_path)
    
    model_wotr = YOLO(wotr_path)
    """
    model_type = "MiDaS_small" 
    model_midas = torch.hub.load("intel-isl/MiDaS", model_type)
    device_midas = torch.device("mps") if torch.cuda.is_available() else torch.device("cpu")
    model_midas.to(device_midas)
    model_midas.eval()
    
    midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")
    if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
        transform_midas = midas_transforms.dpt_transform
    else:
        transform_midas = midas_transforms.small_transform
    """
    model_captioning = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")

def detected_labels_and_boxes_result(model, image):
    results = model.predict(image)
    result = results[0]
    return {'names': result.names, "classes": result.boxes.cls, "boxes": result.boxes.xyxy, "confs": result.boxes.conf}

def depth_map_result(image):
    if isinstance(image, str):
        input_image = cv2.imread(image)
    else:
        input_image = image
    input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
    input_image = transform_midas(input_image).to(device_midas)

    with torch.no_grad():
        depth_prediction = model_midas(input_image)

    depth_map = depth_prediction.squeeze().cpu().numpy()

    depth_map = (depth_map - depth_map.min()) / (depth_map.max() - depth_map.min()) * 255
    depth_map = depth_map.astype(np.uint8)
    return depth_map

def image_captioning_result(image):
    if not isinstance(image, str):
        image = Image.fromarray(np.uint8(image)).convert('RGB')
    return model_captioning(image)

def stereo_vision_distance_result(image_left, image_right, labels_boxes_json):
    classes = labels_boxes_json["classes"]
    boxes = labels_boxes_json["boxes"]
    # TODO 
    # returns distances dict for each boxes
    # example output 
    # {'names': result.names, "classes": result.boxes.cls, "boxes": result.boxes.xyxy, "confs": result.boxes.conf, "distances": distances}
    fl = 75-246.3441162109375*155/490.97529220581055
    image_left = cv2.cvtColor(image_left, cv2.COLOR_BGR2RGB)
    image_right = cv2.cvtColor(image_right, cv2.COLOR_BGR2RGB)
    sz1 = image_right.shape[1]
    sz2 = image_right.shape[0]
    det = labels_boxes_json.boxes
    lbls = labels_boxes_json.classes
    centre = sz1/2
    def get_dist_to_centre_tl(box, cntr = centre):
        pnts = np.array(dist_measurement.tlbr_to_corner(box))[:,0]
        return abs(pnts - centre)
    
    def get_dist_to_centre_br(box, cntr = centre):
        pnts = np.array(dist_measurement.tlbr_to_corner_br(box))[:,0]
        return abs(pnts - centre)
    
    dists_tl =  dist_measurement.get_horiz_dist_corner_tl(det)
    dists_br =  dist_measurement.get_horiz_dist_corner_br(det)
    final_dists = []
    dctl = dist_measurement.get_dist_to_centre_tl(det[0])
    dcbr = dist_measurement.get_dist_to_centre_br(det[0])
    for i, j in zip(*tracks):
        if dctl[i] < dcbr[i]:
            final_dists.append((dists_tl[i][j],lbls[i]))
        else:
            final_dists.append((dists_br[i][j],lbls[i]))
        
        
    tantheta = (1/(155-fl))*(59.0/2)*sz1/246.3441162109375
    fd = [i for (i,j) in final_dists]
    dists_away = (59.0/2)*sz1*(1/tantheta)/np.array(fd)+fl
    cat_dist = []
    for i in range(len(dists_away)):
        cat_dist.append(f'{lbls[i]} {dists_away[i]:.1f}cm')
        print("Estimation:")
        print(f'{lbls[i]} is {dists_away[i]:.1f}cm away')
          

# example usage: get_model_output_from_camera(image_captioning_result, printable=True)
def get_model_output_from_camera(model_method, show=False, printable=False):
    cap = cv2.VideoCapture(0) 
    while True:
        ret, frame = cap.read()  
        if not ret:
            break

        result = model_method(frame)
        if printable:
            print(result)
        if show:
            cv2.imshow("output", result)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        time.sleep(1)

    cap.release()
    cv2.destroyAllWindows()

def get_model_output_from_frame(model_method, frame ,show=False, printable=False):
    result = model_method(frame)
    if printable:
        print(result)
    if show:
        cv2.imshow("output", result)
    cv2.destroyAllWindows()

def main():
    #image_captioning_result(cv2.imread("C:/Users/ege/Desktop/test_images/3/right.jpg"))
    initialize_models()
    print(detected_labels_and_boxes_result(model_yolov8,"C:/Users/ege/Desktop/test_images/3/right.jpg"))
    
if __name__ == "__main__":
    main()




D:\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ege\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
D:\anaconda3\Lib\site-packages\transformers\models\vit\feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(

image 1/1 C:\Users\ege\Desktop\test_images\3\right.jpg: 640x480 1 chair, 1 refrigerator, 2185.1ms
Speed: 125.6ms preprocess, 2185.1ms inference, 232.4ms postprocess per image at shape (1, 3, 640, 480)


{'names': {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 6